In [ ]:
import time
from timeit import default_timer
from contextlib import contextmanager

@contextmanager
def elapsed_timer():
    start = default_timer()
    print 'I\'m here.' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    print 'I\'m done.' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())

In [ ]:
with elapsed_timer() as elapsed:
    time.sleep(4)
    duration = '%.1f' % elapsed()
    print duration
    time.sleep(4)

In [1]:
#Split the all_views, get one file(sentiment, tweet) for each view
#Clean each view
#Run this for each view.
%%bash
fn="tweet_text_clean" #file name is
if [ ! -f "$fn.csv" ]
then
    echo "File: $fn.csv not found"
    exit
fi
#this function will convert text to lowercase and will disconnect punctuation and special symbols from words
function normalize_text {
    awk '{print tolower($0);}' < $1 | sed -e 's/\./ \. /g' -e 's/<br \/>/ /g' -e 's/"/ " /g' \
    -e 's/,/ , /g' -e 's/(/ ( /g' -e 's/)/ ) /g' -e 's/\!/ \! /g' -e 's/\?/ \? /g' \
    -e 's/\;/ \; /g' -e 's/\:/ \: /g' > $1-norm
}
export LC_ALL=C
normalize_text "$fn.csv"
mv "$fn.csv" "$fn-norm"

File: tweet_text_clea.csv not found


In [2]:
import os.path
tw_file = 'tweet_text_clean-norm' #filename-norm
#corp_file = 'tweet_corpus.csv'
assert os.path.isfile(tw_file), tw_file + " unavailable"
#assert os.path.isfile(corp_file), corp_file + " unavailable"

In [22]:
with open(tw_file) as alldata:
    print len(alldata)

TypeError: object of type 'file' has no len()

In [11]:
import gensim
import numpy as np
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

#sentiment = {'positive':1, 'negative':-1} #, 'neutral':2}
sentiment = {'4':1, '0':-1} #- new data 0,4
SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order
#tw_sentiment_dict = {}
#print total_num, train_test_shuffle
train_num = total_num *  7 / 10 # 70% train/test
with open(tw_file) as alldata:
    for line_no, line in enumerate(alldata):
        tokens = gensim.utils.to_unicode(line).split('\t')
        sentiment = tokens[0]
        #if tw_id not in tw_sentiment_dict.keys():
        #    continue
        words = tokens[1]
        split = 'test'
        #sentiment = tw_sentiment_dict[tw_id]
        alldocs.append(SentimentDocument(words, tags, split, sentiment))
#split train/test
total_num = len(alldocs)
train_test_shuffle = np.arange(total_num)
np.random.shuffle(train_test_shuffle)
#set split 
train_docs = []
test_docs = []
for line_no, doc in enumerate(alldocs):
    if train_test_shuffle[line_no] <= train_num:
        doc.split = 'train'
        train_docs.append(doc)
    else:
        test_docs.append(doc)
#train_docs = [doc for doc in alldocs if doc.split == 'train']
#test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

347 docs: 246 train-sentiment, 101 test-sentiment


In [12]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"
model_size = 100
simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=model_size, window=3, negative=5, hs=0, min_count=5, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=model_size, negative=5, hs=0, min_count=5, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=model_size, window=3, negative=5, hs=0, min_count=2, workers=cores),
]

# speed setup by sharing results of 1st model's vocabulary scan
simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template
print(simple_models[0])
for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec(dm/c,d100,n5,w3,mc5,t4)
Doc2Vec(dbow,d100,n5,mc5,t4)
Doc2Vec(dm/m,d100,n5,w3,mc2,t4)


In [13]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])
#print models_by_name['dbow+dmm'], models_by_name['dbow+dmc'] 
#del models_by_name['dbow+dmc']

In [14]:
import numpy as np
import statsmodels.api as sm
from sklearn import svm, metrics
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 
import ipdb

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    #ipdb.set_trace()
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def svm_predictor_from_data(train_targets, train_regressors):
    svc = svm.SVC(kernel='poly', degree=5)
    svc.fit(train_regressors, train_targets)
    return svc

    """expected = svm_y_test
    predicted = svc.predict(svm_x_test)

    #print("Classification report for classifier %s:\n%s\n"
    #      % (svc, metrics.classification_report(expected, predicted)))
    #print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
    """

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    #predictor = logistic_predictor_from_data(train_targets, train_regressors)
    predictor = svm_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    #ipdb.set_trace()
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

In [15]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved

In [16]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        #print name
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list)
            duration = '%.1f' % elapsed()
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

        if ((epoch + 1) % 5) == 0 or epoch == 0:
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if infer_err < best_error[name + '_inferred']:
                best_error[name + '_inferred'] = infer_err
                best_indicator = '*'
            print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2015-11-24 13:54:26.129185
*0.366337 : 1 passes : Doc2Vec(dm/c,d100,n5,w3,mc5,t4) 0.0s 0.0s
*0.200000 : 1 passes : Doc2Vec(dm/c,d100,n5,w3,mc5,t4)_inferred 0.0s 0.0s
*0.366337 : 1 passes : Doc2Vec(dbow,d100,n5,mc5,t4) 0.0s 0.0s
*0.500000 : 1 passes : Doc2Vec(dbow,d100,n5,mc5,t4)_inferred 0.0s 0.0s
*0.366337 : 1 passes : Doc2Vec(dm/m,d100,n5,w3,mc2,t4) 0.0s 0.0s
*0.400000 : 1 passes : Doc2Vec(dm/m,d100,n5,w3,mc2,t4)_inferred 0.0s 0.0s
*0.366337 : 1 passes : dbow+dmm 0.0s 0.0s
*0.400000 : 1 passes : dbow+dmm_inferred 0.0s 0.0s
*0.366337 : 1 passes : dbow+dmc 0.0s 0.0s
*0.400000 : 1 passes : dbow+dmc_inferred 0.0s 0.0s
completed pass 1 at alpha 0.025000
*0.366337 : 2 passes : Doc2Vec(dm/c,d100,n5,w3,mc5,t4) 0.0s 0.0s
*0.366337 : 2 passes : Doc2Vec(dbow,d100,n5,mc5,t4) 0.0s 0.0s
*0.366337 : 2 passes : Doc2Vec(dm/m,d100,n5,w3,mc2,t4) 0.0s 0.0s
*0.366337 : 2 passes : dbow+dmm 0.0s 0.0s
*0.366337 : 2 passes : dbow+dmc 0.0s 0.0s
completed pass 2 at alpha 0.023800
*0.366337 : 3 passes : D

In [17]:
for rate, name in sorted((rate, name) for name, rate in best_error.items()):
    print("%f %s" % (rate, name))

0.100000 dbow+dmc_inferred
0.200000 Doc2Vec(dbow,d100,n5,mc5,t4)_inferred
0.200000 Doc2Vec(dm/c,d100,n5,w3,mc5,t4)_inferred
0.200000 Doc2Vec(dm/m,d100,n5,w3,mc2,t4)_inferred
0.200000 dbow+dmm_inferred
0.366337 Doc2Vec(dbow,d100,n5,mc5,t4)
0.366337 Doc2Vec(dm/c,d100,n5,w3,mc5,t4)
0.366337 Doc2Vec(dm/m,d100,n5,w3,mc2,t4)
0.366337 dbow+dmc
0.366337 dbow+dmm


In [19]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

for doc 66...
Doc2Vec(dm/c,d100,n5,w3,mc5,t4):
 [(281, 0.3276647627353668), (292, 0.2725587487220764), (30, 0.2566581666469574)]
Doc2Vec(dbow,d100,n5,mc5,t4):
 [(281, 0.3276647627353668), (292, 0.2725587487220764), (30, 0.2566581666469574)]
Doc2Vec(dm/m,d100,n5,w3,mc2,t4):
 [(281, 0.3276647627353668), (292, 0.2725587487220764), (30, 0.2566581666469574)]


In [21]:
f = eval('ls')

NameError: name 'ls' is not defined